# Part 3: Creating a Basic Image Agent

![Alt text](img/augLLMs.png)

In this tutorial we'll be making a simplified image classifier/agent with Gemma3.

Theres two parts

* **Multimodal Gemma Classifier** - Using a Gemma model to detect what's in the image and provide a specific output.
* **Downstream Action** - A simple function that can process the results of the action, such as sending an email or anything else!

## Putting it all together

Now that we have an image model ready, let's set up a simple function to interact with the model and its outputs. sLet's redefine `model_call(prompt)` function that sends user input to the LLM and receives a response.  

In [13]:
from ollama import chat
from ollama import ChatResponse
import pprint
from IPython.display import Markdown

image_path = "img/NotHotDog.jpg"  # Replace with the actual path to your image file


def image_chat(prompt, img_path, model="gemma3:27b-it-qat"):
    response = chat(
        model = model, 
        messages=[
            {
                'role': 'user',
                'content': prompt,
                'images': [img_path]
            }
        ]
    )
    return response["message"]["content"]

prompt = "What is this an image of?"
output = image_chat(prompt, image_path)

display(Markdown(output))

This image shows a **dachshund dog dressed in a hot dog costume**. 

The costume is designed to look like a full hot dog with a bun, ketchup, and mustard on top of the dog's back. It's a playful and humorous outfit!

## Hot Dog or not Hotdog Classifier

In [14]:
image_path = "img/NotHotDog.jpg"  # Replace with the actual path to your image file

def image_chat(prompt, img_path, model="gemma3:27b-it-qat"):
    response = chat(
        model = model, 
        messages=[
            {
                'role': 'user',
                'content': prompt,
                'images': [img_path]
            }
        ]
    )
    return response["message"]["content"]

prompt = 'Is this an image of the food item hot dog say yes, otherwise say no, no other output'
image_chat(prompt, image_path) 

'no'

In [15]:
image_path = "img/NotHotDog.jpg"  # Replace with the actual path to your image file

def image_chat(prompt, img_path):
    response = chat(
        model="gemma3:27b-it-qat",  # Use a vision-capable model like LLaVA
        messages=[
            {
                'role': 'user',
                'content': 'Is this an image of the food item hot dog say yes, otherwise say no, no other output',
                'images': [img_path]
            }
        ]
    )
    return response["message"]["content"]


image_chat(None,image_path) 

'no'

## Parsing the response
With our prompt complete we can turn this into a simple classifier. From here you can replace this with any python logic you like, whether its sending an email, or anything else.

In [16]:
image_path = "img/Hot_dog_with_mustard.png"  # Replace with the actual path to your image file

image_chat(None, image_path) 

'yes'

In [17]:
def call_a_tool(img_path):
    """This just prints a string, but it could be anything else"""
    response = image_chat(None, image_path)
    if response.lower() == 'yes':
        return "Give treat"
    return "Add ketchup"

call_a_tool(image_path)

'Give treat'

## Upgrading with Gemini 
If you need a more powerful model it's only one API call away. With this the model stops being local but in turn you get access to frontier capabilities.

In [11]:
from google import genai
from google.genai import types

with open(image_path, 'rb') as f:
      image_bytes = f.read()

def generate():
    client = genai.Client(
        api_key="INSERT_API_KEY",
    )

    # Pick any model from AI studio
    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                    types.Part.from_bytes(
                    data=image_bytes,
                    mime_type='image/jpeg',
                  ),
                types.Part.from_text(text="""What is this?"""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")
generate()

This is a dachshund dog wearing a hot dog costume.

## 🎯 Recap: What We Learned 

In this section, we built our first basic image classification agent that can differentiate between two images and respond accordingly.

Here are the key ideas to remember:
- **Not everything needs to be chat**: Models can be prompted to return parseable outputs quite easily, no architecture changes needed.
- **The model can be an intermediate part of a system**: The model doesn't always need to be front and center of every application.
- **From there you can do anything**: We just outputted strings, but with python (or any other language) we can make our system act agentically do anything.

This pattern — LLM suggests, framework acts — is the foundation for building more complex agents later.